## Setup

In [3]:
import dotenv, os, openai, csv

EXAMPLES_DIR = './examples'
RESULTS_DIR = './results'

dotenv.load_dotenv()
gpt = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def prompt_gpt(prompt: str) -> str:
    response = gpt.chat.completions.create(
        model='gpt-3.5',
        messages=[
            {
                'role': 'user',
                'content': prompt,
            }
        ]
    )
    output = response.choices[0].message.content.strip()
    return output

## Load examples

Data is formatted in CSV files as follows:
```
q_1,a_1
q2,a_2
...
q_n,a_n
```

In [4]:
examples_by_task_class = {}

for filename in os.listdir(EXAMPLES_DIR):
    if filename.endswith('.csv'):
        path = os.path.join(EXAMPLES_DIR, filename)
        
        with open(path, newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            task_class = filename[:-4]
            examples_by_task_class[task_class] = [{'q': row[0], 'a': row[1]} for row in reader]

## Run trials

In [5]:
for task_class, examples in examples_by_task_class.items():
    # case 1: use n / 2 examples
        # case 1a: use the first n / 2 examples
        # case 1b: use the other n / 2 examples
    # case 2: use all n examples
    # case 3: use n / 2 examples, duplicated (for n total examples)
        # case 3a: use the first n / 2 examples
        # case 3b: use the other n / 2 examples
    # other cases
    
    # prompt GPT in each case, accumulate responses and save to output file in RESULTS_DIR for each task_class
    pass